# NBA 3PT shooting data vs touch time

In [2]:
import pandas as pd
import numpy as np
import time
import datetime
from IPython.display import clear_output
from collections import Counter
from functools import reduce
import itertools
from nba_api.stats.static import teams
from pbpstats.client import Client
from nbafuns import *

In [3]:
from nba_api.stats.endpoints import playerdashboardbygeneralsplits as player_fun
from nba_api.stats.endpoints import playerdashptshots as player_fun1

year = 2020
player_dict = get_players(league = 'NBA', from_year = year, to_year = year)

In [4]:
response2 = player_fun1.PlayerDashPtShots(team_id = 0,player_id = '2544',season='2019-20').get_data_frames()
response = player_fun.PlayerDashboardByGeneralSplits(player_id[245],season=['2017-18','2018-19','2019-20']).get_data_frames()


In [9]:
response2[6]

,PLAYER_ID,PLAYER_NAME_LAST_FIRST,SORT_ORDER,GP,G,TOUCH_TIME_RANGE,FGA_FREQUENCY,FGM,FGA,FG_PCT,EFG_PCT,FG2A_FREQUENCY,FG2M,FG2A,FG2_PCT,FG3A_FREQUENCY,FG3M,FG3A,FG3_PCT
0,2544,"James, LeBron",1,67,66,Touch < 2 Seconds,0.249,171,324,0.528,0.620,0.129,111,168,0.661,0.120,60,156,0.385
1,2544,"James, LeBron",2,67,67,Touch 2-6 Seconds,0.312,208,406,0.512,0.564,0.222,166,289,0.574,0.090,42,117,0.359
2,2544,"James, LeBron",3,67,67,Touch 6+ Seconds,0.439,262,571,0.459,0.499,0.322,216,419,0.516,0.117,46,152,0.303


In [10]:
# data = pd.read_csv("players_3PT_FT_filt.csv")
# pID = data["pID"]
pID = []
Player_Name = []

FGM_0_2,    FGA_0_2 = [],[]
FGM_2_6,    FGA_2_6 = [],[]
FGM_6_p,    FGA_6_p = [],[]
FG3M_0_2,   FG3A_0_2 = [],[]
FG3M_2_6,   FG3A_2_6 = [],[]
FG3M_6_p,   FG3A_6_p = [],[]
# len(player_id)
ii = 0
tic = time.perf_counter()
for i in range(len(player_id)):
    time.sleep(0.6)
    pID.append(player_id[i]) 
    response = player_fun1.PlayerDashPtShots(team_id = 0,player_id = player_id[i],season=['2020-21']).get_data_frames()[6]
    response1 = response[response['TOUCH_TIME_RANGE']=='Touch < 2 Seconds']
    response2 = response[response['TOUCH_TIME_RANGE']=='Touch 2-6 Seconds']
    response3 = response[response['TOUCH_TIME_RANGE']=='Touch 6+ Seconds']
    response1 = response1.reset_index(drop=True)
    response2 = response2.reset_index(drop=True)
    response3 = response3.reset_index(drop=True)
    try:
        FGM_0_2.append(response1['FGM'][0])
        FGA_0_2.append(response1['FGA'][0])
        FG3M_0_2.append(response1['FG3M'][0])
        FG3A_0_2.append(response1['FG3A'][0])
    except:
        FGM_0_2.append(0)
        FGA_0_2.append(0)
        FG3M_0_2.append(0)
        FG3A_0_2.append(0)
    try:
        FGM_2_6.append(response2['FGM'][0])
        FGA_2_6.append(response2['FGA'][0])
        FG3M_2_6.append(response2['FG3M'][0])
        FG3A_2_6.append(response2['FG3A'][0])
    except:
        FGM_2_6.append(0)
        FGA_2_6.append(0)
        FG3M_2_6.append(0)
        FG3A_2_6.append(0)
    try:
        FGM_6_p.append(response3['FGM'][0])
        FGA_6_p.append(response3['FGA'][0])
        FG3M_6_p.append(response3['FG3M'][0])
        FG3A_6_p.append(response3['FG3A'][0])
    except:
        FGM_6_p.append(0)
        FGA_6_p.append(0)
        FG3M_6_p.append(0)
        FG3A_6_p.append(0)
    Player_Name.append([player['Name'] for player in player_dict if player['pID'] == player_id[i]])
    clear_output(wait=True)
    print(i+1)
    ii+=1
    print(ii)
toc = time.perf_counter()
print((toc-tic)/60)

546
546
14.227764233700002


In [12]:
Player_Name = list(itertools.chain(*Player_Name))
dataset = pd.DataFrame({'pID':pID,'Player_Name':Player_Name,
'FGM_0_2': FGM_0_2, 'FGA_0_2': FGA_0_2, 
'FGM_2_6': FGM_2_6, 'FGA_2_6': FGA_2_6,
'FGM_6_p': FGM_6_p, 'FGA_6_p': FGA_6_p,
'FG3M_0_2': FG3M_0_2, 'FG3A_0_2': FG3A_0_2,
'FG3M_2_6': FG3M_2_6, 'FG3A_2_6': FG3A_2_6,
'FG3M_6_p': FG3M_6_p, 'FG3A_6_p': FG3A_6_p
})
dataset.to_csv ('players_shooting_touchtime_2020_21.csv',index=False)


In [13]:
# print(dataset)
dataset.iloc[0:3,:]

,pID,Player_Name,FGM_0_2,FGA_0_2,FGM_2_6,FGA_2_6,FGM_6_p,FGA_6_p,FG3M_0_2,FG3A_0_2,FG3M_2_6,FG3A_2_6,FG3M_6_p,FG3A_6_p
0,1630173,Precious Achiuwa,94,166,30,61,0,1,0,1,0,0,0,0
1,1629121,Jaylen Adams,0,2,1,4,0,2,0,1,0,0,0,1
2,203500,Steven Adams,149,229,36,71,3,7,0,2,0,1,0,0
